In [ ]:
import pandas as pd
import numpy as np

In [ ]:
anime = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Raw/anime.csv')
rating = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Raw/rating.csv')

In [ ]:
anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [ ]:
print("Old shape: ", rating.shape)
anime_id = anime['anime_id'].unique()
print("anime_id len: ", len(anime_id))

rated_anime_id = rating['anime_id'].unique()
print("rated_anime_id len: ", len(rated_anime_id))

unrated_anime_id = []
for aid in anime_id:
    if aid not in rated_anime_id:
        unrated_anime_id.append(aid)
unrated_anime_id = np.asarray(unrated_anime_id)
print("unrated_anime_id len: ", len(unrated_anime_id))

strange_anime_id = []
for aid in rated_anime_id:
    if aid not in anime_id:
        strange_anime_id.append(aid)
strange_anime_id = np.asarray(strange_anime_id)
print("strange_anime_id len: ", len(strange_anime_id))

print("Delete strange anime in rating dataframe:")
rating = rating[(rating.anime_id != strange_anime_id[0]) & (rating.anime_id != strange_anime_id[1]) & (rating.anime_id != strange_anime_id[2])]
rated_anime_id = rating['anime_id'].unique()
print("rated_anime_id len: ", len(rated_anime_id))
strange_anime_id = []
for aid in rated_anime_id:
    if aid not in anime_id:
        strange_anime_id.append(aid)
strange_anime_id = np.asarray(strange_anime_id)
print("New shape: ", rating.shape)


Old shape:  (7813737, 3)
anime_id len:  12294
rated_anime_id len:  11200
unrated_anime_id len:  1097
strange_anime_id len:  3
Delete strange anime in rating dataframe:
rated_anime_id len:  11197
New shape:  (7813727, 3)


In [ ]:
user_id = rating['user_id'].unique()
print("user_id len: ", len(user_id))

user_id len:  73515


user_id is from 1 to 73516 except 46809.

Just care about two set: rated_anime_id (length of 11197) and user_id (length of 73151)

In [ ]:
anime['genre'] = anime['genre'].fillna('')
genre_set = set()
for genres in anime['genre']:
    genres = genres.split(', ')
    genre_set.update(genres)
genre_set.remove('')
print(type(genre_set))
genre_list = sorted(genre_set)
# print(len(genre_set))
# for genre in genre_set:
#     print(genre)

genre_index_dict = {}
for i in range(len(genre_list)):
    genre_index_dict[genre_list[i]] = i
print(genre_index_dict)
print(type(genre_list))


def encode_genre(genres):
    result = np.asarray([0] * len(genre_list))
    if genres == '':
        return result
    genres = genres.split(', ')
    for genre in genres:
        result[genre_index_dict[genre]] = 1
    return result


anime['encoded_genre'] = anime['genre'].apply(lambda x: list(encode_genre(x)))
print(anime.columns)
print(anime[['anime_id', 'encoded_genre']])

In [ ]:
anime[['anime_id', 'encoded_genre']]

In [ ]:
rated_anime_id = rating['anime_id'].unique()
len(rated_anime_id)

11197

In [ ]:
anime['type'].unique()

array(['Movie', 'TV', 'OVA', 'Special', 'Music', 'ONA', nan], dtype=object)

In [ ]:
anime[(anime['type'] == 'Movie') & (anime['episodes'] == 'Unknown')]

,anime_id,name,genre,type,episodes,rating,members,encoded_genre
8142,30617,Animegatari,Comedy,Movie,Unknown,5.36,1308,"[0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10940,33820,Ao Oni The Animation (Movie),Horror,Movie,Unknown,NaN,1416,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ..."
10947,32086,Blame! Movie,"Action, Drama, Mecha, Psychological, Sci-Fi, S...",Movie,Unknown,NaN,3251,"[1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
10996,33783,Godzilla,"Action, Adventure, Fantasy, Sci-Fi",Movie,Unknown,NaN,1022,"[1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ..."


In [ ]:
TYPE_DICT = {
    'Movie'     : [1, 0, 0, 0, 0, 0],
    'TV'        : [0, 1, 0, 0, 0, 0],
    'OVA'       : [0, 0, 1, 0, 0, 0],
    'Special'   : [0, 0, 0, 1, 0, 0],
    'Music'     : [0, 0, 0, 0, 1, 0],
    'ONA'       : [0, 0, 0, 0, 0, 1],
    ''          : [0, 0, 0, 0, 0, 0]
}
anime['encoded_type'] = anime['type'].fillna('').apply(lambda x: TYPE_DICT[x])

In [ ]:
anime['encoded_type']

0        [1, 0, 0, 0, 0, 0]
1        [0, 1, 0, 0, 0, 0]
2        [0, 1, 0, 0, 0, 0]
3        [0, 1, 0, 0, 0, 0]
4        [0, 1, 0, 0, 0, 0]
                ...        
12289    [0, 0, 1, 0, 0, 0]
12290    [0, 0, 1, 0, 0, 0]
12291    [0, 0, 1, 0, 0, 0]
12292    [0, 0, 1, 0, 0, 0]
12293    [1, 0, 0, 0, 0, 0]
Name: encoded_type, Length: 12294, dtype: object

In [ ]:
anime['type'] = anime['type'].fillna('')

In [ ]:
s_episodes_by_type = {}
for t in anime['type'].unique().tolist():
    if t != '':
        s_episodes_by_type[t] = {'count': 0, 'sum': 0}
s_episodes_by_type['All'] = {'count': 0, 'sum': 0}
s_episodes_by_type

{'All': {'count': 0, 'sum': 0},
 'Movie': {'count': 0, 'sum': 0},
 'Music': {'count': 0, 'sum': 0},
 'ONA': {'count': 0, 'sum': 0},
 'OVA': {'count': 0, 'sum': 0},
 'Special': {'count': 0, 'sum': 0},
 'TV': {'count': 0, 'sum': 0}}

In [ ]:
for i in range(len(anime)):
    if anime['episodes'].iloc[i] != 'Unknown':
        s_episodes_by_type['All']['count'] += 1
        s_episodes_by_type['All']['sum'] += int(anime['episodes'].iloc[i])
        if anime['type'].iloc[i] != '':
            s_episodes_by_type[anime['type'].iloc[i]]['count'] += 1
            s_episodes_by_type[anime['type'].iloc[i]]['sum'] += int(anime['episodes'].iloc[i])
s_episodes_by_type

{'All': {'count': 11954, 'sum': 148021},
 'Movie': {'count': 2344, 'sum': 2584},
 'Music': {'count': 487, 'sum': 551},
 'ONA': {'count': 613, 'sum': 4216},
 'OVA': {'count': 3261, 'sum': 7884},
 'Special': {'count': 1671, 'sum': 4280},
 'TV': {'count': 3578, 'sum': 128506}}

In [ ]:
def fill_episodes(type_name, episodes):
    if episodes == 'Unknown':
        if type_name == '':
            return float(s_episodes_by_type['All']['sum'] / s_episodes_by_type['All']['count'])
        else:
            return float(s_episodes_by_type[type_name]['sum'] / s_episodes_by_type[type_name]['count'])
    else:
        return float(episodes)

In [ ]:
anime['filled_episodes'] = anime.apply(lambda row: fill_episodes(row['type'], row['episodes']), axis=1)

In [ ]:
anime

,anime_id,name,genre,type,episodes,rating,members,encoded_genre,encoded_type,filled_episodes
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0]",1.0
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]",64.0
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0]",51.0
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]",24.0
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0]",51.0
...,...,...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",1.0
12290,5543,Under World,Hentai,OVA,1,4.28,183,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",1.0
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",4.0
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",1.0


In [ ]:
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaler1 = MinMaxScaler()
anime[['members']] = scaler1.fit_transform(anime[['members']])

In [ ]:
scaler2 = MinMaxScaler()
anime[['filled_episodes']] = scaler2.fit_transform(anime[['filled_episodes']])

In [ ]:
anime['filled_episodes']

0        0.000000
1        0.034673
2        0.027518
3        0.012658
4        0.027518
           ...   
12289    0.000000
12290    0.000000
12291    0.001651
12292    0.000000
12293    0.000000
Name: filled_episodes, Length: 12294, dtype: float64

In [ ]:
anime

,anime_id,name,genre,type,episodes,rating,members,encoded_genre,encoded_type,filled_episodes
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,0.197872,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0]",0.000000
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,0.782770,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]",0.034673
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,0.112689,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0]",0.027518
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,0.664325,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]",0.012658
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,0.149186,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0]",0.027518
...,...,...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,0.000203,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",0.000000
12290,5543,Under World,Hentai,OVA,1,4.28,0.000176,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",0.000000
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,0.000211,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",0.001651
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,0.000168,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",0.000000


In [ ]:
anime = anime[anime['anime_id'].isin(rated_anime_id)]

In [ ]:
anime

,anime_id,name,genre,type,episodes,rating,members,encoded_genre,encoded_type,filled_episodes
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,0.197872,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ...","[1, 0, 0, 0, 0, 0]",0.000000
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,0.782770,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]",0.034673
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,0.112689,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0]",0.027518
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,0.664325,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 1, 0, 0, 0, 0]",0.012658
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,0.149186,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ...","[0, 1, 0, 0, 0, 0]",0.027518
...,...,...,...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,0.000203,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",0.000000
12290,5543,Under World,Hentai,OVA,1,4.28,0.000176,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",0.000000
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,0.000211,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",0.001651
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,0.000168,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ...","[0, 0, 1, 0, 0, 0]",0.000000


In [ ]:
import ast
def merge_fields(encoded_genre, encoded_type, filled_episodes, members):
    result = encoded_genre + encoded_type + [filled_episodes] + [members]
    return result

In [ ]:
anime['long_vector'] = anime.apply(lambda row: merge_fields(row['encoded_genre'], row['encoded_type'], row['filled_episodes'], row['members']), axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
anime['short_vector'] = anime['encoded_genre']

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
row0 = anime.iloc[0]

In [ ]:
len(row0['short_vector']), len(row0['long_vector'])

(43, 51)

In [ ]:
anime[['anime_id', 'short_vector']].to_csv('/content/gdrive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Encode/short_vector.csv', index=False)

In [ ]:
pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Encode/short_vector.csv')

,anime_id,short_vector
0,32281,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,5114,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,28977,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,9253,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,9969,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...
11192,9316,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
11193,5543,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
11194,5621,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
11195,6133,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [ ]:
anime[['anime_id', 'long_vector']].to_csv('/content/gdrive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Encode/long_vector.csv', index=False)

In [ ]:
pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Encode/long_vector.csv')

,anime_id,long_vector
0,32281,"[0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,5114,"[1, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, ..."
2,28977,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
3,9253,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,9969,"[1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, ..."
...,...,...
11192,9316,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
11193,5543,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
11194,5621,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
11195,6133,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."


In [ ]:
rating.to_csv('/content/gdrive/MyDrive/Colab Notebooks/Anime Recommendation System/Data/Rating/enable_rating.csv', index=False)